In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True)

In [3]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [4]:
trainPath = "../Dataset/Car damage/body/training"
testPath = "../Dataset/Car damage/body/validation"
training_set=train_datagen.flow_from_directory(trainPath,
                                               target_size=(224,224),
                                               batch_size=10,
                                               class_mode='categorical')
test_set=test_datagen.flow_from_directory(testPath,
                                               target_size=(224,224),
                                               batch_size=10,
                                               class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [28]:
trainPath1 = "../Dataset/Car damage/level/training"
testPath1 = "../Dataset/Car damage/level/validation"
training_set1=train_datagen.flow_from_directory(trainPath1,
                                               target_size=(224,224),
                                               batch_size=10,
                                               class_mode='categorical')
test_set1=test_datagen.flow_from_directory(testPath1,
                                               target_size=(224,224),
                                               batch_size=10,
                                               class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [6]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [7]:
vgg=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))
vgg1=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [8]:
for layer in vgg.layers:
    layer.trainable=False
x=Flatten()(vgg.output)
for layer in vgg1.layers:
    layer.trainable=False
y=Flatten()(vgg1.output)

In [9]:
prediction=Dense(3,activation='softmax')(x)
prediction1=Dense(3,activation='softmax')(y)

In [10]:
model=Model(inputs=vgg.input,outputs=prediction)
model1=Model(inputs=vgg1.input,outputs=prediction1)

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])

In [12]:
import sys
r=model.fit_generator(training_set,
                      validation_data=test_set,
                      epochs=25,
                      steps_per_epoch=979//10,
                      validation_steps=171//10)

C:\Users\ELCOT\AppData\Local\Temp\ipykernel_5900\3143213416.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r=model.fit_generator(training_set,


Epoch 1/25
97/97 [==============================] - 1677s 17s/step - loss: 1.1566 - acc: 0.5686 - val_loss: 0.8822 - val_acc: 0.6353
Epoch 2/25
97/97 [==============================] - 1653s 17s/step - loss: 0.6687 - acc: 0.7544 - val_loss: 0.8208 - val_acc: 0.6824
Epoch 3/25
97/97 [==============================] - 1638s 17s/step - loss: 0.4845 - acc: 0.8204 - val_loss: 1.0894 - val_acc: 0.6353
Epoch 4/25
97/97 [==============================] - 1635s 17s/step - loss: 0.3835 - acc: 0.8586 - val_loss: 1.1416 - val_acc: 0.6529
Epoch 5/25
97/97 [==============================] - 1636s 17s/step - loss: 0.2441 - acc: 0.9174 - val_loss: 1.1818 - val_acc: 0.6471
Epoch 6/25
97/97 [==============================] - 1634s 17s/step - loss: 0.2237 - acc: 0.9216 - val_loss: 1.0819 - val_acc: 0.6471
Epoch 7/25
97/97 [==============================] - 1639s 17s/step - loss: 0.2243 - acc: 0.9278 - val_loss: 1.1719 - val_acc: 0.6588
Epoch 8/25
97/97 [==============================] - 1636s 17s/step - 

In [15]:
model.save('body.h5')


In [20]:
from tensorflow.keras.models import  load_model
import cv2
from skimage.transform import resize

body_model=load_model('body.h5')
def detect(frame):
    img=cv2.resize(frame,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if(np.max(img)>1):
        img=img/255.0
    img=np.array([img])
    prediction =body_model.predict(img)
    print(prediction)
    label=["front","rear","side"]
    preds=label[np.argmax(prediction)]
    return preds


In [27]:
import numpy as np

data="../accident.jpeg"
image=cv2.imread(data)
print(detect(image))

1/1 [==============================] - 2s 2s/step
[[0.9522021  0.04156308 0.00623486]]
front


In [29]:
import sys
r1= model1.fit_generator(training_set1,
                        validation_data=test_set1,
                        epochs=25,
                        steps_per_epoch=979//10,
                        validation_steps=171//10)
model1.save('level.h5')


C:\Users\ELCOT\AppData\Local\Temp\ipykernel_5900\1933596904.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r1= model1.fit_generator(training_set1,


Epoch 1/25
97/97 [==============================] - 1718s 17s/step - loss: 1.1678 - acc: 0.5593 - val_loss: 0.9057 - val_acc: 0.6118
Epoch 2/25
97/97 [==============================] - 1664s 17s/step - loss: 0.7936 - acc: 0.7038 - val_loss: 1.2184 - val_acc: 0.5588
Epoch 3/25
97/97 [==============================] - 1647s 17s/step - loss: 0.5799 - acc: 0.7761 - val_loss: 1.2300 - val_acc: 0.5765
Epoch 4/25
97/97 [==============================] - 1637s 17s/step - loss: 0.3953 - acc: 0.8504 - val_loss: 1.5192 - val_acc: 0.5412
Epoch 5/25
97/97 [==============================] - 1631s 17s/step - loss: 0.3721 - acc: 0.8555 - val_loss: 1.5730 - val_acc: 0.5235
Epoch 6/25
97/97 [==============================] - 1618s 17s/step - loss: 0.2836 - acc: 0.8947 - val_loss: 1.1682 - val_acc: 0.6471
Epoch 7/25
97/97 [==============================] - 1629s 17s/step - loss: 0.2330 - acc: 0.9143 - val_loss: 1.1455 - val_acc: 0.6235
Epoch 8/25
97/97 [==============================] - 1637s 17s/step - 

In [30]:
level_model = load_model('level.h5')


def detect1(frame):
    img = cv2.resize(frame, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if(np.max(img) > 1):
        img = img/255.0
    img = np.array([img])
    prediction = level_model.predict(img)
    print(prediction)
    label = ["minor", "moderate", "severe"]
    preds = label[np.argmax(prediction)]
    return preds


In [31]:
data = "../0111.JPEG"
image = cv2.imread(data)
print(detect1(image))


1/1 [==============================] - 3s 3s/step
[[6.039188e-07 5.539544e-02 9.446039e-01]]
severe
